In [1]:
import sys
sys.path.append("../")
from utils import *

In [2]:
with open(data_path + "train_test_data.pkl", "rb") as f:
    X_train, y_train, X_test, y_test = pickle.load(f)
    
# Convert to lists
y_train = list(y_train.values())
print(len(y_train))
y_test = list(y_test.values())
print(len(y_test))

7473
2525


In [3]:
d = {'pl':pd.read_csv(data_path+'pl_tracks.csv'),\
     't':pd.read_csv(data_path+'tracks.csv')}

In [4]:
# Calculate popularity: number of occurances
pop = d['pl'].groupby('track_uri').count().reset_index()
pop.columns = ['track_uri', 'popularity']
df = pd.merge(d['t'], pop, on = 'track_uri', how = 'left')
df.head()

,track_name,duration_ms,track_uri,artist_uri,album_uri,popularity
0,Lose Control (feat. Ciara & Fat Man Scoop),226863,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,spotify:artist:2wIVse2owClT7go1WT98tk,spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,6841
1,Toxic,198800,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,spotify:artist:26dSoYclwsYLMAKD3tpOr4,spotify:album:0z7pVBGOD7HCIB7S8eLkLI,13205
2,Crazy In Love,235933,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,spotify:artist:6vWDO969PvNqNYHIOW5v0m,spotify:album:25hVFAxTlDvXbx2X2QkUkE,16914
3,Rock Your Body,267266,spotify:track:1AWQoqb9bSvzTjaLralEkT,spotify:artist:31TPClRtHm23RisEBtV3X7,spotify:album:6QPkyl04rXwTGlGlcYaRoW,9066
4,It Wasn't Me,227600,spotify:track:1lzr43nnXAijIGYnCT8M8H,spotify:artist:5EvFsr3kj42KNv97ZEnqij,spotify:album:6NmFmPX56pcLBOFMhIiKvF,23173


In [5]:
# Define the list of track_uri ordered by popularity
pop_sorted_tracks = df.sort_values('popularity', ascending = False)['track_uri'].values.tolist()

In [6]:
def popularity_recommender(data_train, test_size = 10):
    ''' A naive recommender system that outputs the most popular <test_size> songs
        that is not currently in the album. '''
    preds = []
    for pid,visited in data_train.items():
        cur_preds = []
        for track in pop_sorted_tracks:
            if track not in visited:
                cur_preds.append(track)
            if len(cur_preds) == test_size:
                preds.append(cur_preds)
                break
    return preds

In [7]:
y_pred_train = popularity_recommender(X_train)
y_pred_test = popularity_recommender(X_test)

In [8]:
avg_overlap_score_train = np.mean([overlap_score(a,b) for a,b in zip(y_pred_train, y_train)])
avg_overlap_score_test = np.mean([overlap_score(a,b) for a,b in zip(y_pred_test, y_test)])

print(f"Average overlap score for training is {avg_overlap_score_train:.6f}")
print(f"Average overlap score for testing is {avg_overlap_score_test:.6f}")

Average overlap score for training is 0.103840
Average overlap score for testing is 0.100198


In [9]:
avg_dcg_score_train = np.mean([dcg(a,b) for a,b in zip(y_pred_train, y_train)])
avg_dcg_score_test = np.mean([dcg(a,b) for a,b in zip(y_pred_test, y_test)])

print(f"Average DCG score for training is {avg_dcg_score_train:.6f}")
print(f"Average DCG score for testing is {avg_dcg_score_test:.6f}")

Average DCG score for training is 0.050165
Average DCG score for testing is 0.048477
